In [5]:
# FIRST: Let's import necessary libraries for handling files and downloading data
# Floowing that step we are going to study the dataset
import os
import tarfile
import urllib

# Define constants for dataset location
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("dataset","housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# Function to fetch and extract the housing dataset
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """
    Downloads the housing dataset from the given URL and extracts it into the specified directory.
    If the directory does not exist, it is created.
    """
    os.makedirs(housing_path, exist_ok=True)  # Ensure the directory exists
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)  # Download the file
    housing_tgz = tarfile.open(tgz_path)  # Open the tar file
    housing_tgz.extractall(path=housing_path)  # Extract contents
    housing_tgz.close()  # Close the file

In [6]:
# Call the function to download the data
fetch_housing_data()

In [10]:
# Import pandas for data handling
import pandas as pd

# Function to load the dataset into a Pandas DataFrame
def load_housing_data(housing_path=HOUSING_PATH):
    """
    Reads the housing dataset CSV file and returns it as a Pandas DataFrame.
    """
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)  # Load and return the data

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load the dataset into a DataFrame
housing = load_housing_data()

# Display the first few rows of the dataset to verify successful loading
housing.head()

In [ ]:
# Display information regarding the dataset
housing.info()

In [ ]:
# Count the amount of values contain inside the "ocean_proximity" attribute
housing["ocean_proximity"].value_counts()

In [ ]:
# Display a description of the numerical attributes of the dataset
housing.describe()

In [ ]:
# Import Matplotlib for data visualization
import matplotlib.pyplot as plt

# Generate histograms for numerical attributes to understand their distribution
housing.hist(bins=50, figsize=(20,15))
plt.show()  # Display the histograms

In [ ]:
# SECOND: We are going to generate a train set
import numpy as np

# Function to split the dataset into training and testing sets
def split_train_test(data, test_ratio):
    """
    Splits the dataset into a training set and a test set based on the given test ratio.
    Uses a fixed random seed to ensure consistency across runs.
    """
    np.random.seed(42)  # Ensures reproducibility of the random shuffling
    shuffled_indices = np.random.permutation(len(data))  # Generate shuffled indices
    test_set_size = int(len(data) * test_ratio)  # Determine test set size
    test_indices = shuffled_indices[:test_set_size]  # Select test indices
    train_indices = shuffled_indices[test_set_size:]  # Select train indices
    return data.iloc[train_indices], data.iloc[test_indices]  # Return the split datasets

In [ ]:
# Split the dataset into training and test sets with a 20% test size
train_set, test_set = split_train_test(housing, 0.2)

In [ ]:
# Check the size of the training set
len(train_set)

In [ ]:
# Check the size of the test set
len(test_set)

In [ ]:
from zlib import crc32  # Importing crc32 from zlib for checksum calculation

def test_set_check(identifier, test_ratio):  
    """
    Determines whether a given identifier should be part of the test set.

    Uses CRC32 hashing to ensure consistent splitting of data. 
    This is useful to maintain the same test set even if new data is added later.

    Parameters:
    identifier (int): A unique identifier for a data instance.
    test_ratio (float): Proportion of the dataset to assign to the test set.

    Returns:
    bool: True if the instance should be in the test set, False otherwise.
    """
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):  
    """
    Splits a dataset into training and test sets based on a unique identifier.

    Ensures that the test set remains stable even if new data instances are added.
    Uses `test_set_check` to consistently determine which rows belong to the test set.

    Parameters:
    data (DataFrame): The dataset to split.
    test_ratio (float): The proportion of data to assign to the test set.
    id_column (str): The column name containing unique identifiers.

    Returns:
    tuple: (training set DataFrame, test set DataFrame)
    """
    ids = data[id_column]  # Extract unique identifiers
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))  # Apply the hash-based test set check
    return data.loc[~in_test_set], data.loc[in_test_set]  # Return train and test sets


In [ ]:
housing_with_id = housing.reset_index() # adds and "index" column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index") # uses index as a indentifier, if new datasets are added, the must be appended

In [ ]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id") # uses longitude and latitude as unique identifiers

In [ ]:
from sklearn.model_selection import train_test_split  

# Perform a random train-test split on the housing dataset.
# This method is more flexible than `split_train_test_by_id`, as it allows:
# - Setting a random seed for reproducibility (`random_state=42`).
# - Splitting multiple datasets simultaneously.
# - Optional stratification to maintain class distributions.
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)  


In [ ]:
# We generate income categories based on the "median_income" attribute.
# This is done to perform stratified sampling later, ensuring that the test set 
# contains a proportional representation of different income levels.

# Stratified sampling helps reduce bias in our test set by ensuring that all income
# groups are well-represented, avoiding issues where rare income levels might be underrepresented.

housing["income_cat"] = pd.cut(housing["median_income"],  # Categorizing "median_income"
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],  # Define income brackets
                               labels=[1, 2, 3, 4, 5])  # Assign category labels

# Display a histogram of the newly created income categories to visualize distribution
housing["income_cat"].hist()


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit  

# Create an instance of StratifiedShuffleSplit to ensure that the test set 
# maintains the same proportion of income categories as the full dataset.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Generate new train and test sets using stratified sampling based on the "income_cat" attribute.
for train_index, test_index in split.split(housing, housing["income_cat"]):  
    strat_train_set = housing.loc[train_index]  # Extract stratified training set
    strat_test_set = housing.loc[test_index]  # Extract stratified test set


In [ ]:
# Compute the proportion of each income category in the stratified test set.
# This verifies that the stratified sampling worked correctly by maintaining 
# the same distribution of "income_cat" as in the original dataset.

strat_test_set["income_cat"].value_counts() / len(strat_test_set)


In [ ]:
# Remove the "income_cat" column from both the training and test sets.
# This is done because "income_cat" was only created for stratified sampling 
# and is not an original feature of the dataset.

for set_ in (strat_train_set, strat_test_set):  
    set_.drop("income_cat", axis=1, inplace=True)  # Drop the column in place


In [ ]:
# THIRD, we are going to explore the data further

# Create a copy of the stratified training set for data exploration.
# This ensures that any transformations or modifications made during exploration
# do not affect the original training data.

housing = strat_train_set.copy()  # Copy the training set to explore it safely


In [ ]:
# Generate a scatter plot to visualize the geographical distribution of data points.
# The x-axis represents longitude, and the y-axis represents latitude.
# This helps identify spatial patterns in the dataset, such as clustering of houses.

housing.plot(kind="scatter", x="longitude", y="latitude")


In [ ]:
# Generate a scatter plot of housing data points based on geographic coordinates.
# The alpha parameter (alpha=0.1) makes points more transparent, helping to reveal 
# dense regions where data points overlap.
# This allows us to better visualize population density and housing distribution.

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)


In [ ]:
# Generate a scatter plot to visualize housing data, including:
# - The geographic coordinates (longitude, latitude)
# - The size of points proportional to population (scaled by 100 for better visualization)
# - The color of points representing median house values, with a color map for better interpretation

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,  # Plot with some transparency
            s=housing["population"]/100,  # Size of points scaled by population size
            label="population",  # Add a label for population (used in legend)
            figsize=(10,7),  # Set figure size for better readability
            c="median_house_value",  # Use median house value to color the points
            cmap=plt.get_cmap("jet"),  # Apply a color map to represent house values
            colorbar=True,  # Show color bar to indicate house value range
            )

# Display the legend to explain the population label
plt.legend()


In [ ]:
# Display the first rows of the dataset
housing.head()

In [ ]:
# Remove the 'ocean_proximity' column from the dataset as it contains categorical string data,
# which cannot be directly correlated with numerical values (floats). This ensures we only have 
# numerical features that can be used in machine learning models.

housing = housing.drop(columns=['ocean_proximity'])  # Drop the 'ocean_proximity' column
housing.head()  # Display the first few rows of the updated dataset to confirm the change


In [ ]:
# Compute the correlation matrix for all numerical features in the dataset.
# The correlation matrix shows how each feature is linearly related to the others,
# with values ranging from -1 (perfect negative correlation) to 1 (perfect positive correlation).
# Values close to 0 indicate little to no linear relationship.

corr_matrix = housing.corr()  # Calculate the correlation matrix for the housing dataset
print(corr_matrix)  # Display the correlation matrix to explore relationships between features


In [ ]:
# Compute and sort the correlation of all features with 'median_house_value' in descending order
# This helps identify which features have the strongest positive or negative correlation with the target variable
corr_matrix["median_house_value"].sort_values(ascending=False)  


In [ ]:
from pandas.plotting import scatter_matrix

# Define a list of attributes to explore their relationships through scatter plots
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]

# Create a scatter matrix to visually explore correlations between the selected attributes.
# Diagonal histograms show the distribution of individual attributes, while off-diagonal scatter plots 
# show pairwise correlations between attributes. A correlation of 1 results in a diagonal histogram instead of a plot.
scatter_matrix(housing[attributes], figsize=(12,8))  

In [ ]:
# Generate a scatter plot to visualize the relationship between median_income and median_house_value.
# The alpha parameter (alpha=0.1) makes the points semi-transparent to help reveal areas with dense data points.
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)  

In [ ]:
# Generate new useful attributes to improve the dataset:
# - rooms_per_household: The number of rooms per household
# - bedrooms_per_room: The proportion of bedrooms to total rooms
# - population_per_household: The number of people per household

housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

# Recalculate the correlation matrix to see how the newly created features relate to median_house_value
corr_matrix = housing.corr()

# Sort the correlation values for 'median_house_value' in descending order to examine relationships
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# FOURTH: Clean the dataset by removing the target variable ('median_house_value') from the features
# This allows us to work with a clean dataset containing only predictors (input features).

housing = strat_train_set.drop("median_house_value", axis=1)  # Drop the target variable from the features
housing_labels = strat_train_set["median_house_value"].copy()  # Store the target variable separately


In [ ]:
# Start by addressing the missing values in the 'total_bedrooms' attribute.
# There are three options for handling missing values:

# Option 1: Remove the rows where 'total_bedrooms' has missing values.
housing.dropna(subset=["total_bedrooms"])  # Remove rows with missing 'total_bedrooms'

# Option 2: Drop the 'total_bedrooms' attribute from the dataset entirely.
housing.drop("total_bedrooms", axis=1)  # Drop the 'total_bedrooms' column

# Option 3: Fill the missing values with the median of 'total_bedrooms'.
median = housing["total_bedrooms"].median()  # Calculate the median of 'total_bedrooms'
housing["total_bedrooms"].fillna(median, inplace=True)  # Replace missing values with the median

In [ ]:
from sklearn.impute import SimpleImputer

# Create an imputer object that will fill missing values with the median of each attribute.
imputer = SimpleImputer(strategy="median")

# Remove the 'ocean_proximity' column as it's a categorical attribute and can't be processed by the imputer.
housing_num = housing.drop("ocean_proximity", axis=1)

# Fit the imputer on the numerical features of the dataset, which will compute the median value for each attribute.
imputer.fit(housing_num)

In [ ]:
# Access the statistics (i.e., median values) computed by the imputer for each numerical attribute.
# These values are stored in the 'statistics_' attribute after fitting the imputer.
imputer.statistics_  # Display the median values for each attribute

In [ ]:
# Display the median values of each numerical attribute in the housing_num dataset.
# This is done using pandas' median() method, which computes the median for each column.
housing_num.median().values  # Show the median values for each numerical attribute

In [ ]:
# Use the imputer to fill missing values in housing_num with the computed median values.
# The result is returned as a numpy array with the missing values replaced.
X = imputer.transform(housing_num)  # Fill missing values and return as a numpy array

# Convert the numpy array (X) back to a pandas DataFrame, preserving the original column names and indices.
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)  # Convert to DataFrame

In [ ]:
# Extract the 'ocean_proximity' column, which is the only categorical attribute in the dataset.
housing_cat = housing[["ocean_proximity"]]  # Select the 'ocean_proximity' column

# Display the first 10 values of the 'ocean_proximity' attribute to inspect its content.
housing_cat.head(10)  # Show the first 10 rows of the categorical data

In [ ]:
# Import the OrdinalEncoder from Scikit-learn to convert categorical values to numerical values.
from sklearn.preprocessing import OrdinalEncoder

# Create an OrdinalEncoder object to transform the categorical 'ocean_proximity' into numerical values.
ordinal_encoder = OrdinalEncoder()

# Apply the OrdinalEncoder to the 'ocean_proximity' column and store the result in housing_cat_encoded.
# The result is a numpy array with the encoded values for each category.
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

# Display the first 10 encoded values to inspect the transformation.
housing_cat_encoded[:10]  # Show the first 10 rows of the encoded values

In [ ]:
# Access the categories that were assigned numerical values by the OrdinalEncoder.
# This shows the mapping of the original categorical values to their corresponding numerical labels.
ordinal_encoder.categories_  # Display the categories and their assigned numerical labels

In [ ]:
# ML algorithms assume that numerical values with similar proximity are more related. 
# One solution is to use one-hot encoding, which creates a separate binary attribute for each category. 
# Each attribute will be 1 when the category is present, and 0 otherwise.

from sklearn.preprocessing import OneHotEncoder

# Create an instance of OneHotEncoder to perform the one-hot encoding.
cat_encoder = OneHotEncoder()

# Apply the OneHotEncoder to the 'ocean_proximity' column to transform it into binary columns.
# This will return a sparse matrix where each category is represented by a separate column with 0s and 1s.
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

# Display the resulting sparse matrix after the one-hot encoding.
housing_cat_1hot  # One-hot encoded sparse matrix for 'ocean_proximity'

In [ ]:
housing_cat_1hot.toarray() # changes from a sparse matrix into a numpy array

In [ ]:
# Convert the one-hot encoded sparse matrix into a dense numpy array for easier inspection.
# This will return a 2D numpy array where each row represents a sample and each column represents one category.
housing_cat_1hot.toarray()  # Convert the sparse matrix to a numpy array

In [ ]:
# Now we will create a custom transformer by subclassing BaseEstimator and TransformerMixin.
# This will allow us to add additional features to the dataset.

from sklearn.base import BaseEstimator, TransformerMixin

# Define column indices for the features we will use in the transformation.
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

# Custom transformer class to add new attributes to the dataset.
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    # Constructor to control whether to add 'bedrooms_per_room' or not.
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room  # Default is True, can be set to False
    
    # Fit method (nothing to fit, so it just returns 'self').
    def fit(self, X, y=None):
        return self  # No fitting needed
    
    # Transform method that computes new attributes and adds them to the dataset.
    def transform(self, X):
        # Calculate rooms per household: divide rooms by households for each row
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        
        # Calculate population per household: divide population by households for each row
        population_per_household = X[:, population_ix] / X[:, households_ix]
        
        # If we want to add 'bedrooms_per_room', calculate it and add it to the dataset.
        if self.add_bedrooms_per_room:
            # Calculate bedrooms per room: divide bedrooms by rooms for each row
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            # Return a new array with the added attributes (rooms_per_household, population_per_household, and optionally bedrooms_per_room)
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            # Return a new array with only rooms_per_household and population_per_household
            return np.c_[X, rooms_per_household, population_per_household]

# Create an instance of the custom transformer without adding 'bedrooms_per_room'.
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)

# Apply the transformer to the housing dataset to generate new features.
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
# Generate a pipeline that:
# 1. Fills missing values with the median of each feature (SimpleImputer).
# 2. Combines new attributes (e.g., rooms_per_household, population_per_household) using the custom transformer (CombinedAttributesAdder).
# 3. Scales the features to have zero mean and unit variance using StandardScaler.

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create a pipeline for numerical data preprocessing
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),  # Step 1: Handle missing values by replacing them with the median
    ('attribs_adder', CombinedAttributesAdder()),   # Step 2: Add new attributes like rooms_per_household and population_per_household
    ('std_scaler', StandardScaler()),               # Step 3: Scale the features to standardize them
])

# Apply the pipeline to the numerical features (housing_num) to preprocess them.
# The result is stored in housing_num_tr and contains the transformed features.
# Note: housing_num does not include 'ocean_proximity' or 'median_house_value', as 'ocean_proximity' is categorical and 'median_house_value' is the target variable.
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
from sklearn.compose import ColumnTransformer

# Define the list of numerical attributes (excluding the target variable 'median_house_value')
num_attribs = list(housing_num)  

# Define the list of categorical attributes (in this case, only 'ocean_proximity')
cat_attribs = ["ocean_proximity"]

# Create a full pipeline to process both numerical and categorical attributes:
full_pipeline = ColumnTransformer([
    # Step 1: Apply the 'num_pipeline' to all numerical attributes (defined earlier).
    # This includes imputing missing values, adding new features, and scaling.
    ("num", num_pipeline, num_attribs),
    
    # Step 2: Apply OneHotEncoder to the categorical attribute 'ocean_proximity'.
    # This converts it into binary columns (one for each category).
    ("cat", OneHotEncoder(), cat_attribs),
])

# Apply the full pipeline to the dataset (excluding 'median_house_value').
# The pipeline will transform the numerical and categorical parts of the dataset accordingly,
# and then the outputs will be concatenated into a single array.
housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
# FIFTH: Train a machine learning model using the prepared dataset.

from sklearn.linear_model import LinearRegression

# Create an instance of the LinearRegression model.
lin_reg = LinearRegression()

# Fit the model to the prepared data (features) and the target labels (housing_labels).
# The model will learn the relationship between the features in housing_prepared and the target variable (median_house_value).
lin_reg.fit(housing_prepared, housing_labels)  # Train the model using the prepared dataset and the labels

In [ ]:
# Select a small subset of the dataset (first 5 rows) to verify the model's predictions.
some_data = housing.iloc[:5]  # Select the first 5 rows of the dataset for testing
some_labels = housing_labels.iloc[:5]  # Select the corresponding labels (median_house_value) for the first 5 rows

# Apply the full pipeline transformation to the selected subset of data.
# This ensures that the data is preprocessed (imputed, transformed, and scaled) in the same way as the training data.
some_data_prepared = full_pipeline.transform(some_data)

# Use the trained linear regression model to predict the median house values for the selected data.
# Print the predictions made by the model.
print("Predictions:", lin_reg.predict(some_data_prepared))

# Print the actual labels (ground truth) for comparison with the predictions.
print("Labels:", list(some_labels))

In [ ]:
# Calculate the Root Mean Squared Error (RMSE) to evaluate the performance of the model.

from sklearn.metrics import mean_squared_error

# Use the trained linear regression model to make predictions on the entire prepared housing dataset.
housing_predictions = lin_reg.predict(housing_prepared)

# Calculate the Mean Squared Error (MSE) by comparing the predicted values with the actual labels (housing_labels).
lin_mse = mean_squared_error(housing_labels, housing_predictions)

# Calculate the Root Mean Squared Error (RMSE) by taking the square root of the MSE.
lin_rmse = np.sqrt(lin_mse)

# Output the RMSE value, which provides an indication of the model's prediction accuracy.
lin_rmse

In [ ]:
# Since the linear regression model has a large error of USD 68k, we will switch to a more powerful model: DecisionTreeRegressor.

from sklearn.tree import DecisionTreeRegressor

# Create an instance of the DecisionTreeRegressor model.
tree_reg = DecisionTreeRegressor()

# Train the DecisionTreeRegressor model using the prepared dataset and the housing labels (median_house_value).
tree_reg.fit(housing_prepared, housing_labels)  # Fit the model to the training data

In [ ]:
# Now, we will calculate the RMSE for the DecisionTreeRegressor model to evaluate its performance.

# Use the trained DecisionTreeRegressor model to make predictions on the entire prepared housing dataset.
housing_predictions = tree_reg.predict(housing_prepared)

# Calculate the Mean Squared Error (MSE) by comparing the predicted values with the actual labels (housing_labels).
tree_mse = mean_squared_error(housing_labels, housing_predictions)

# Calculate the Root Mean Squared Error (RMSE) by taking the square root of the MSE.
tree_rmse = np.sqrt(tree_mse)

# Output the RMSE value, which will give an indication of how well the DecisionTreeRegressor model is performing.
tree_rmse

In [ ]:
# Since a 0% error is too good to be true, it suggests that the model might be overfitting.
# To assess the model's performance more reliably, we will use K-fold cross-validation.
# In K-folds cross-validation, the training set is split into 10 "folds", and the model is trained 10 times,
# each time using 9 folds for training and 1 fold for validation. This helps to reduce overfitting.

from sklearn.model_selection import cross_val_score

# Perform 10-fold cross-validation on the DecisionTreeRegressor model.
# The cross_val_score function computes the performance of the model by training and evaluating it on each fold.
# The scoring parameter is set to "neg_mean_squared_error" to return the negative of the mean squared error (so higher scores are better).
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

# Convert the negative mean squared error scores into positive values, then take the square root to calculate RMSE for each fold.
tree_rmse_scores = np.sqrt(-scores)

# Output the RMSE scores for each fold of the cross-validation.
tree_rmse_scores

In [ ]:
# Define a function to display the performance scores of the model.
# This function takes the array of RMSE scores from cross-validation and prints:
# 1. The individual scores for each fold.
# 2. The mean of the scores (average RMSE across folds).
# 3. The standard deviation of the scores (how much the scores vary across folds).

def display_scores(scores):
    print("Scores:", scores)  # Print the RMSE scores for each fold.
    print("Mean:", scores.mean())  # Print the mean RMSE score (average error).
    print("Standard deviation:", scores.std())  # Print the standard deviation of RMSE scores.

# Call the function to display the RMSE scores of the DecisionTreeRegressor after cross-validation.
display_scores(tree_rmse_scores)

In [ ]:
# Since the DecisionTreeRegressor model has a higher RMSE score than the linear regression model,
# let's perform cross-validation on the linear regression model to evaluate its performance more reliably.

# Perform 10-fold cross-validation on the linear regression model.
# The cross_val_score function computes the performance of the model by training and evaluating it on each fold.
# The scoring parameter is set to "neg_mean_squared_error" to return the negative of the mean squared error (so higher scores are better).
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

# Convert the negative mean squared error scores into positive values, then take the square root to calculate RMSE for each fold.
lin_rmse_scores = np.sqrt(-lin_scores)

# Display the RMSE scores for the linear regression model after cross-validation.
display_scores(lin_rmse_scores)

In [ ]:
# Since the DecisionTreeRegressor model was overfitting and performed worse than linear regression,
# let's try using a RandomForestRegressor, which is an ensemble model that can handle overfitting better.

from sklearn.ensemble import RandomForestRegressor

# Create an instance of the RandomForestRegressor model.
forest_reg = RandomForestRegressor()

# Train the RandomForestRegressor model using the prepared dataset and the housing labels (median_house_value).
forest_reg.fit(housing_prepared, housing_labels)

# Make predictions on the entire prepared housing dataset using the trained RandomForestRegressor model.
housing_predictions = forest_reg.predict(housing_prepared)

# Calculate the Mean Squared Error (MSE) by comparing the predicted values with the actual labels (housing_labels).
forest_mse = mean_squared_error(housing_labels, housing_predictions)

# Calculate the Root Mean Squared Error (RMSE) by taking the square root of the MSE.
forest_rmse = np.sqrt(forest_mse)

# Output the RMSE value, which will give an indication of how well the RandomForestRegressor model is performing.
forest_rmse

In [ ]:
# Now, let's perform cross-validation on the RandomForestRegressor to evaluate its performance more reliably.

# Perform 10-fold cross-validation on the RandomForestRegressor model.
# The cross_val_score function computes the performance of the model by training and evaluating it on each fold.
# The scoring parameter is set to "neg_mean_squared_error" to return the negative of the mean squared error (so higher scores are better).
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

# Convert the negative mean squared error scores into positive values, then take the square root to calculate RMSE for each fold.
forest_rmse_scores = np.sqrt(-forest_scores)

# Display the RMSE scores for the RandomForestRegressor model after cross-validation.
display_scores(forest_rmse_scores)

In [ ]:
# Now we will save the trained models for future reference, so we can load and use them without retraining.

import joblib

# Save the trained models to disk using joblib. This will store the models in .pkl files.
joblib.dump(lin_reg, "lin_reg.pkl")  # Save the linear regression model
joblib.dump(tree_reg, "tree_reg.pkl")  # Save the decision tree regressor model
joblib.dump(forest_reg, "forest_reg.pkl")  # Save the random forest regressor model

# You can load the saved models later using joblib.load(). Example:
# my_model_loaded = joblib.load("my_model.pkl")  # Load a saved model from the file

In [ ]:
# SIXTH: Fine-tune the models using SciKit-Learn's GridSearchCV to find the best hyperparameters.

from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the GridSearchCV.
# The grid search will try all combinations of 'n_estimators' and 'max_features' for the first dictionary,
# and 'bootstrap', 'n_estimators', and 'max_features' for the second dictionary.
param_grid = [ 
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},  # First set of parameters to test.
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}  # Second set of parameters to test.
]

# Create an instance of the RandomForestRegressor model.
forest_reg = RandomForestRegressor()

# Create the GridSearchCV object, specifying the model, parameter grid, and evaluation settings.
# - cv=5: 5-fold cross-validation
# - scoring='neg_mean_squared_error': to use negative mean squared error as the evaluation metric
# - return_train_score=True: to store and return the training scores for each hyperparameter combination.
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring='neg_mean_squared_error',
                          return_train_score=True)

# Fit the grid search object on the training data.
# It will search through all the combinations of hyperparameters, train the model, and evaluate it using cross-validation.
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
# After fitting the grid search, we can retrieve the best hyperparameters and the best model.

# Get the best combination of hyperparameters found by GridSearchCV.
# This will return the hyperparameters that gave the best performance based on the cross-validation.
grid_search.best_params_

In [ ]:
# Get the best estimator (model) based on the best combination of hyperparameters.
# This will return the trained model with the best hyperparameters.
grid_search.best_estimator_

In [ ]:
# After performing GridSearchCV, we can inspect the results and evaluate how different hyperparameter combinations performed.

# Access the cross-validation results from the grid search.
cvres = grid_search.cv_results_

# Loop through the results and print the RMSE for each hyperparameter combination.
# We take the negative of the mean test score (since it was computed as negative MSE), 
# then convert it back to positive RMSE by taking the square root.
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)  # Print the RMSE (square root of negative MSE) and the associated hyperparameters.

# You can compare these scores with the previous Random Forest model to see if the grid search gives a better result.
# For example, if the best score is 49,666 (lower than the previous 50,491), it shows that the model is improving,
# but it may still not be ideal.

In [ ]:
# After finding the best model using GridSearchCV, we can inspect the feature importances.
# Feature importance tells us how important each feature is in making predictions.

# Access the feature importances from the best model found by the grid search.
feature_importances = grid_search.best_estimator_.feature_importances_

# Print the feature importances to see how much each feature contributes to the model's predictions.
feature_importances

In [ ]:
# Now, let's display the feature importances along with their corresponding attribute names.

# Define the extra attributes we created (not part of the original dataset).
extra_attribs = ["rooms_per_household", "pop_per_hhold", "bedrooms_per_room"]

# Get the list of categorical attributes that were one-hot encoded in the pipeline.
# The full_pipeline named transformer 'cat' gives access to the OneHotEncoder that was applied to the categorical column.
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])  # Extract the one-hot encoded category names.

# Combine the numerical attributes, extra attributes, and one-hot encoded category attributes into a single list of all feature names.
attributes = num_attribs + extra_attribs + cat_one_hot_attribs

# Zip the feature importances with the corresponding attribute names and sort them in descending order to see the most important features first.
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
# SEVENTH: Evaluate the final model using the test set to estimate its performance on unseen data.

# Get the best model from the grid search (final model after hyperparameter tuning).
final_model = grid_search.best_estimator_

# Prepare the test data: drop the target variable ("median_house_value") from the features.
X_test = strat_test_set.drop("median_house_value", axis=1)

# Extract the target variable ("median_house_value") from the test set for comparison.
y_test = strat_test_set["median_house_value"].copy()

# Transform the test features using the same preprocessing pipeline used for the training set.
X_test_prepared = full_pipeline.transform(X_test)

# Make predictions on the test set using the final model.
final_predictions = final_model.predict(X_test_prepared)

# Calculate the Mean Squared Error (MSE) to evaluate the performance of the model on the test set.
final_mse = mean_squared_error(y_test, final_predictions)

# Calculate the Root Mean Squared Error (RMSE) by taking the square root of the MSE.
final_rmse = np.sqrt(final_mse)

# Print the final RMSE value to evaluate how well the model performs on the test set.
final_rmse

In [ ]:
# To determine how precise the model's predictions are, we can calculate a confidence interval for the RMSE.

from scipy import stats

# Set the confidence level (95% in this case).
confidence = 0.95

# Compute the squared errors by subtracting the true values from the predicted values, then squaring the result.
squared_errors = (final_predictions - y_test) ** 2

# Use scipy.stats.t.interval() to calculate the confidence interval for the RMSE.
# The t.interval() function computes a confidence interval based on the t-distribution.
# - len(squared_errors) - 1: degrees of freedom (sample size - 1)
# - loc: the mean of the squared errors
# - scale: the standard error of the mean (sem) of the squared errors
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))